In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import matplotlib.pyplot as plt

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
Y = df_train['Survived']

## Overview

In [ ]:
df_train[:10]

## Histograms

In [ ]:
df_train.hist(bins=50, figsize=(20, 15))
plt.show()

## Information on columns

In [ ]:
df_train.describe()

## Build of my Transformers

In [ ]:
#Custom Transformer which drop colunmns
from sklearn.base import TransformerMixin
class DropTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        cat=['Ticket', 'Fare', 'Cabin', 'Name', 'SibSp', 'Parch', 'PassengerId']
        Xnew = X.drop(cat, axis=1)
        return Xnew

In [ ]:
#Custom Transformer get dummies of my categorical columns
class GetDummiesTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        features_totransform = ['Sex', 'Embarked', 'Pclass', 'Title']
        for var in features_totransform:
            X = pd.concat([X.drop(var, axis=1), pd.get_dummies(X[var], prefix=var)], axis=1)
        return X

In [ ]:
#Custom Transformer for impute median age into NaN values
class Imputer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['Age'] = X['Age'].fillna(df_train['Age'].median())
        return X


In [ ]:
#Custom Transformer for extract Title of people
class Title(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for i in X:
            X['Title']=X['Name'].str.extract('([A-Za-z]+)\.', expand=False)  # Use REGEX to define a search pattern
        return X

In [ ]:
#Custom Transformer obtains the number of person of the same family
class Family(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for i in X:
            X['Family']=X['SibSp'] + X['Parch']
        return X

## My Pipelines

In [ ]:
#Pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([ 
        ('imp', Imputer()),
        ('title', Title()),
        ('family', Family()),
        ('drop', DropTransformer()),
        ('encoder', GetDummiesTransformer())
    ])

df_train_tr = pipeline.fit_transform(df_train)
df_test_tr = pipeline.transform(df_test)



df_train_tr contrains 28 columns wheareas df_test_tr contains only 19 columns..

A lot of Titles are not given

In [ ]:
#Custom Transformer for modify the Title into other caterogies
class TitleTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for i in X:
            X['Title']=X['Title'].replace(['Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev'], 'Officer')
            X['Title']=X['Title'].replace(['Lady', 'Countess', 'Sir', 'Jonkheer', 'Dona', 'Master'], 'Royalty')
            X['Title']=X['Title'].replace('Mlle', 'Miss')
            X['Title'] = X['Title'].replace('Ms', 'Miss')
            X['Title'] = X['Title'].replace('Mme', 'Mrs')
            
        return X

#Pipeline
from sklearn.pipeline import Pipeline
pipeline2= Pipeline([ 
        ('imp', Imputer()),
        ('title', Title()),
        ('family', Family()),
        ('drop', DropTransformer()),
        ('titlet', TitleTransformer()),
        ('encoder', GetDummiesTransformer())
    ])

df_train_tr = pipeline2.fit_transform(df_train)
df_test_tr = pipeline2.transform(df_test)


## Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV

X_train = df_train_tr.drop(['Survived'], axis=1).values
X_test = df_test_tr
y_train_1 = (df_train['Survived'] == 1)

SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)


sgd_clf.fit(X_train, y_train_1)





In [ ]:
Some_data = X_train[10]
Some_data_predict = sgd_clf.predict([Some_data])
print('Is he survived ? ', Some_data_predict, 'real issue is ', Y[10])



Accuracy on samples

In [ ]:
#Calcul of the exact rate : more than 90% ! But there's only 1 / 10 to have a 3
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_1, cv=3, scoring="accuracy")

Confusion matrix

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_1, cv=3)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_1, y_train_pred)

In [ ]:
#Precision / recall
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_1, y_train_pred)#0.39492753623188404 39% true when he said "Survived"
recall_score(y_train_1, y_train_pred)#0.956140350877193 95% of the Survived are covered

#Combo between precision & recall : F1
from sklearn.metrics import f1_score
f1_score(y_train_1, y_train_pred)

Configure Threshold to optimize results

In [ ]:
from sklearn.metrics import precision_recall_curve

y_scores = cross_val_predict(sgd_clf, X_train, y_train_1, cv=3,
                             method="decision_function")
precisions, recalls, thresholds = precision_recall_curve(y_train_1, y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.axvline(x=2000,color='red')
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])   
    
plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.xlim([-2000, 5000])
plt.show()

Best parameters for SGDClassifier

In [ ]:
y_train_pred_90 = (y_scores > 2000)
print(precision_score(y_train_1, y_train_pred_90))

print(recall_score(y_train_1, y_train_pred_90))

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_1, y_scores)
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
plt.show()

Final Score

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_1, y_scores)

RandomForest Classifier

In [ ]:
from  sklearn.ensemble  import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)

## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}

grid_search = GridSearchCV(forest_clf,param_grid = rf_param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)

grid_search.fit(X_train, y_train_1)

forest_clf = grid_search.best_estimator_
print(grid_search.best_params_)


y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_1, cv=3,
                                    method="predict_proba")


In [ ]:
#Calcul of the exact rate : more than 76% ! better than the previous
from sklearn.model_selection import cross_val_score
cross_val_score(forest_clf, X_train, y_train_1, cv=3, scoring="accuracy")

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(forest_clf, X_train, y_train_1, cv=3)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_1, y_train_pred)

In [ ]:
#Precision / recall
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_1, y_train_pred)#0.728125 73% des fois où il dit Survived : c'est vrai
recall_score(y_train_1, y_train_pred)#0.6812865497076024 68% de la totalité des Survived a été couverte

#On peut combiner precision_score avec le recall_score en une métrique unique : F1
from sklearn.metrics import f1_score
f1_score(y_train_1, y_train_pred)

In [ ]:
from sklearn.metrics import roc_curve


y_scores_forest = y_probas_forest[:, 1] # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_1, y_scores_forest)



In [ ]:
from sklearn.metrics import precision_recall_curve

precisions_forest, recalls_forest, thresholds_forest = precision_recall_curve(y_train_1, y_scores_forest)

def plot_precision_recall_vs_threshold(precisions_forest, recalls_forest, thresholds_forest):
    plt.plot(thresholds_forest, precisions_forest[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds_forest, recalls_forest[:-1], "g-", label="Recall", linewidth=2)
    plt.axvline(x=0.43,color='red')
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])

plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions_forest, recalls_forest, thresholds_forest)
plt.xlim([-1, 2])
plt.show()

In [ ]:
y_train_pred_90 = (y_scores_forest > 0.43)
print(precision_score(y_train_1, y_train_pred_90))

print(recall_score(y_train_1, y_train_pred_90))

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
roc_auc_score(y_train_1, y_scores_forest)

In [ ]:
forest_clf.fit(X_train, y_train_1)
y_final = forest_clf.predict_proba(X_test)
y_final = pd.DataFrame(y_final)

predict_mine = np.where(y_final > 0.43, 1, 0)
predict_mine = pd.DataFrame(predict_mine)
predict_mine = predict_mine[1]

In [ ]:
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": predict_mine
    })
submission.to_csv("Forest.csv", index=False)

Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gradient_clf = GradientBoostingClassifier(random_state=42)

gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

grid_search = GridSearchCV(gradient_clf,param_grid = gb_param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)

grid_search.fit(X_train,y_train_1)

gradient_clf = grid_search.best_estimator_

print(grid_search.best_params_)

y_probas_gradient = cross_val_predict(gradient_clf, X_train, y_train_1, cv=3,
                                    method="predict_proba")


In [ ]:
cross_val_score(gradient_clf, X_train, y_train_1, cv=3, scoring="accuracy")
#Seems to be better than the RandomForest

In [ ]:
y_train_pred = cross_val_predict(gradient_clf, X_train, y_train_1, cv=3)

confusion_matrix(y_train_1, y_train_pred)

In [ ]:
#Precision / recall
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_1, y_train_pred)#0.8235294117647058 82% des fois où il dit Survived : c'est vrai
recall_score(y_train_1, y_train_pred)#0.695906432748538 69% de la totalité des Survived a été couverte

f1_score(y_train_1, y_train_pred)

In [ ]:
y_scores_gradient = y_probas_gradient[:, 1] # score = proba of positive class
fpr_gradient, tpr_gradient, thresholds_gradient = roc_curve(y_train_1, y_scores_gradient)

from sklearn.metrics import precision_recall_curve
precisions_gradient, recalls_gradient, thresholds_gradient = precision_recall_curve(y_train_1, y_scores_gradient)

def plot_precision_recall_vs_threshold(precisions_gradient, recalls_gradient, thresholds_gradient):
    plt.plot(thresholds_gradient, precisions_gradient[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds_gradient, recalls_gradient[:-1], "g-", label="Recall", linewidth=2)
    plt.axvline(x=0.38,color='red')
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])

plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions_gradient, recalls_gradient, thresholds_gradient)
plt.xlim([0, 1])
plt.show()

In [ ]:
y_train_pred_90 = (y_scores_gradient > 0.38)
print(precision_score(y_train_1, y_train_pred_90))

print(recall_score(y_train_1, y_train_pred_90))

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plot_roc_curve(fpr_gradient, tpr_gradient, "Gradient Boosting")
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
roc_auc_score(y_train_1, y_scores_gradient)

In [ ]:
gradient_clf.fit(X_train, y_train_1)
y_final = gradient_clf.predict_proba(X_test)
y_final = pd.DataFrame(y_final)

predict_mine = np.where(y_final > 0.38, 1, 0)
predict_mine = pd.DataFrame(predict_mine)
predict_mine = predict_mine[1]

In [ ]:
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": predict_mine
    })
submission.to_csv("Gradient.csv", index=False)

AdaBoostClassifier


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

DTC = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(DTC, random_state=42)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

grid_search = GridSearchCV(ada_clf,param_grid = ada_param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)

grid_search.fit(X_train,y_train_1)

ada_clf = grid_search.best_estimator_

print(grid_search.best_params_)

y_probas_ada = cross_val_predict(ada_clf, X_train, y_train_1, cv=3,
                                    method="predict_proba")


In [ ]:
cross_val_score(ada_clf, X_train, y_train_1, cv=3, scoring="accuracy")
#Seems to be better than the RandomForest

Doesn't seems to be better than Gradient Boosting

In [ ]:
y_train_pred = cross_val_predict(ada_clf, X_train, y_train_1, cv=3)

confusion_matrix(y_train_1, y_train_pred)

In [ ]:
#Precision / recall
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_1, y_train_pred)#0.7470588235294118 74% des fois où il dit Survived : c'est vrai
recall_score(y_train_1, y_train_pred)#0.7426900584795322 74% de la totalité des Survived a été couverte
f1_score(y_train_1, y_train_pred)

In [ ]:
y_scores_ada = y_probas_ada[:, 1] # score = proba of positive class
fpr_ada, tpr_ada, thresholds_adat = roc_curve(y_train_1, y_scores_ada)

from sklearn.metrics import precision_recall_curve
precisions_ada, recalls_ada, thresholds_ada = precision_recall_curve(y_train_1, y_scores_ada)

def plot_precision_recall_vs_threshold(precisions_ada, recalls_ada, thresholds_ada):
    plt.plot(thresholds_ada, precisions_ada[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds_ada, recalls_ada[:-1], "g-", label="Recall", linewidth=2)
    plt.axvline(x=0.49,color='red')
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])

plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions_ada, recalls_ada, thresholds_ada)
plt.xlim([0, 1])
plt.show()

In [ ]:
y_train_pred_90 = (y_scores_ada > 0.49)
print(precision_score(y_train_1, y_train_pred_90))

print(recall_score(y_train_1, y_train_pred_90))

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plot_roc_curve(fpr_gradient, tpr_gradient, "Gradient Boosting")
plot_roc_curve(fpr_ada, tpr_ada, "AdaBoost")
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
roc_auc_score(y_train_1, y_scores_ada)

In [ ]:
ada_clf.fit(X_train, y_train_1)
y_final = ada_clf.predict_proba(X_test)
y_final = pd.DataFrame(y_final)

predict_mine = np.where(y_final > 0.50, 1, 0)
predict_mine = pd.DataFrame(predict_mine)
predict_mine = predict_mine[1]

In [ ]:
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": predict_mine
    })
submission.to_csv("Ada.csv", index=False)

Voting

In [ ]:

from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[
        ('lr', forest_clf), ('rf', gradient_clf), ('adb', ada_clf)], voting='soft')
eclf1 = eclf1.fit(X_train, y_train_1)
predictions=eclf1.predict(X_train)
predictions

test_predictions=eclf1.predict(X_test)

test_predictions=test_predictions.astype(int)
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": test_predictions
    })

submission.to_csv("titanic_submission.csv", index=False)